### Decision Tree - 决策树模型

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()   # 导入鸢尾花数据集

X = iris.data[:, 2:] # petal length and width - 花瓣长度和宽度
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)

tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [3]:
from sklearn.tree import export_graphviz

export_graphviz(
            tree_clf,
            out_file="./images/iris_tree.dot",
            feature_names=iris.feature_names[2:],
            class_names=iris.target_names,
            rounded=True,
            filled=True
)

!dot -Tpng ./images/iris_tree.dot -o ./images/iris_tree.png

使用 `$ dot -Tpng iris_tree.dot -o iris_tree.png` 可将 `.dot` 文件转换为 png 格式

结果如下（先运行上面的单元格，生成 iris_tree.png）：

<img src="./images/iris_tree.png" width=450>

节点的 samples 属性统计该节点用于多少个训练样本；value 属性表示该节点对于每个类别有多少个样本；gini 属性用于衡量数据的不纯度。

> 数据的不纯度越低，同质性越高。
>
> 如果一个节点包含的所有训练样本都是同一类的，则 gini = 0

Gini Impurity:

$$G_i = 1 - \sum_{k = 1}^{n}{P_{i, k}^2}$$

其中 $P_{i, k}$ 表示第 i 个节点中，属于第 k 类的样本所占的比例。

> Scikit-Learn 库使用的是 CART 算法，该算法只产生二叉树。然而，ID3 等算法可以产生超过两个字节点的决策树模型

---

#### 白盒与黑盒：

正如我们看到的一样，决策树非常直观，他们的决定很容易被解释。这种模型通常被称为白盒模型。

相反，随机森林或神经网络通常被认为是黑盒模型。他们能做出很好的预测，并且可以轻松检查它们做出这些预测过程中计算的执行过程。

然而，人们通常很难用简单的术语来解释为什么模型会做出这样的预测。例如，如果一个神经网络说一个特定的人出现在图片上，我们很难知道究竟是什么导致了这一个预测的出现：模型是否认出了那个人的眼睛？她的嘴？她的鼻子？她的鞋？或者是否坐在沙发上？

相反，决策树提供良好的、简单的分类规则，甚至可以根据需要手动操作（例如鸢尾花分类）。

In [4]:
# 决策树还可以估计某个样本属于特定类别的概率
print(tree_clf.predict_proba([[5, 1.5]]))

print(tree_clf.predict([[5, 1.5]]))  # 输出概率最大的类别

[[0.         0.90740741 0.09259259]]
[1]


#### CART 训练算法

Scikit-Learn 使用分类回归树（Classification And Regression Tree）算法训练决策树。该算法的思想如下：

首先寻找使得划分后的子集 Gini 系数最小的特征-阈值对 $(k, t_k)$，使用该特征 $k$ 和阈值 $t_k$ 划分训练集。

这一过程通过最小化如下 cost function 实现：

$$J(k, t_k) = \frac{m_{\text{left}}}{m}G_{\text{left}} + \frac{m_{\text{right}}}{m}G_{\text{right}}$$

其中，$m$ 是样本个数，$G$ 为 Gini 系数。

算法成功将训练集划分为两部分之后，会递归地对子集进行划分。当到达预定的最大深度（由 `max_depth` 超参数决定）或找不到可以继续降低 Gini 系数的划分方法后停止分裂。

> 除了 `max_depth` 之外，还有一些超参数可以控制决策树的增长条件，如：`min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_leaf_nodes` 等。

CART 算法是一种贪心算法，其贪心地搜索最优划分方式，然后在每一层重复该过程。它不检查是否能够在多层中的所有划分组合中找到最优方案。

贪心算法通常会产生一个局部最优解，但不保证是全局最优。

找到最优的决策树是一个 NP 完全问题。因此，我们需要设计一个 “合理的”（而不是最佳的）解决方案。

#### 计算复杂度

在建立好决策树模型后，做出预测需要遍历决策树。决策树通常近似为一个平衡树，因此遍历决策树需要访问 $O(\log_{2}{m})$ 个节点。

由于每个节点只需要检查一个特征的值，因此总体的预测复杂度仅为 $O(\log_{2}{m})$，与特征数量无关。所以在处理大型训练集时，预测速度也非常快。

然而，训练算法时需要比较所有特征（如果设置了 `max_features` 会少一些）。因此，在每个节点的训练复杂度为 $O(nm\log_{2}{m})$。

对于小型训练集（样本少于几千行），Scikit-learn 可以通过设置 `presort=True` 来加速训练。但是这对于较大的训练集来说会显著减慢训练速度。

#### 纯度指标

##### 1. 信息增益（Information Gain）

信息熵（Information Entropy）一般用于衡量随机变量的不确定性，其定义式如下：

$$\operatorname{Ent}(D) = - \sum_{i = 1}^{N}{p_i \log_{2}{p_i}}$$

其中 $D$ 表示样本集， $N$ 表示样本集分类数，$p_i$ 表示第 $i$ 类样本在样本集所占比例。

> 随机变量的不确定性越大，则纯度越低；即信息熵越大，纯度越低。

对于一个离散变量而言，可以根据其在样本集中的所有可能取值将原样本集划分为子样本集。

衡量一个划分的好坏的标准是划分前后信息熵的减少量，如果信息熵减少得越多，则不确定性下降得越多。

这种划分的度量指标称为信息增益（Information Gain），其定义式如下：

$$\operatorname{Gain}(D, k) = \operatorname{Ent}(D) - \sum_{j = 1}^{M} \frac{\left| D_j \right|}{|D|} \operatorname{Ent}\left( D_j \right)$$

其中 $D$ 表示样本集，$k$ 表示离散特征，$M$ 表示离散特征 $k$ 所有可能取值的数量，$D_j$ 表示样本集中第 $j$ 种取值的子样本集。

> 一般而言，并不会将样本集按照特征的所有可能取值进行划分，因为这种划分对于小样本集而言，可能导致划分之后的子集过小。
>
> 其次，这种划分还可能导致子集数量过多，且对于不同的特征，划分的子集数量不定，生成的决策树在维护上较为复杂。
>
> 一般而言，决策树在中间节点的测试是对单一特征的单一阈值进行的，因此会在所有可能取值中，找到使得划分前后信息增益最大的特征的阈值。

当特征是连续变量时，其取值不像离散变量那样是有限的。这时可以将连续变量在样本集中的所有可能取值排序后俩俩取平均值作为划分点，改写上式。

当只检查一个特征的一个阈值时，定义式如下：

$$\begin{align*}
T_k &= \left\{ \frac{k_i + k_{i + 1}}{2} | 1 \le i \le M - 1 \right\} \\
\operatorname{Gain}(D, k) &= \max_{t \in T_k} \operatorname{Gain}(D, k, t) \\
&= \max_{t \in T_k} \left( \operatorname{Ent}(D) − \frac{|D_{\text{left}}^t|}{|D|} \operatorname{Ent}(D_{\text{left}}^t) − \frac{|D_{\text{right}}^t|}{|D|} \operatorname{Ent}(D_{\text{right}}^t) \right)
\end{align*}
$$

其中，$T_k$ 表示平均值的集合，$D_{\text{left/right}}^t$ 表示左右节点拥有的样本集（左侧为小于阈值，右侧大于阈值）。

划分的目标是使得样本集按照某个特征的某个阈值划分之后的纯度提高，因此可找到最合适的划分特征及阈值，如下：

$$k_{\text{best}}, t_{\text{best}} = \underset{k, t}{\operatorname{argmax}} \operatorname{Gain}(D, k, t)$$

##### 2. 基尼指数（Gini Index）

基尼系数（Gini Coefficient）一般用于衡量数据的不纯度，基尼系数的值越小，数据的纯度越高，同质性越高。其定义式如下：

$$\operatorname{Gini}(D) = 1- \sum_{k = 1}^{K} p_k^2$$

其中 $D$ 表示样本集，$K$ 表示样本集分类数，$p_k$ 表示第 $k$ 类样本在样本集所占比例。

与信息增益中一致，离散特征可以根据其取值将样本集划分为不同的子集。定义基尼指数（Gini Index）为子集基尼系数的加权平均，其一般用于衡量一个随机选中的样本被错误分类的概率。因此，基尼指数越大，划分效果越差。

基尼系数的定义式如下：

$$\operatorname{Gini}(D, k)= \sum_{j = 1}^{M} \frac{\left| D^{j} \right|}{|D|} \operatorname{Gini} \left(D^{j} \right)$$

其中 $D$ 表示样本集，$k$ 表示离散属性，$M$ 表示离散属性 $k$ 所有可能取值的数量，$D^j$ 表示样本集中第 $j$ 种取值的子样本集。

对于连续型属性，将样本集排序后俩俩取均值作为划分点，改写上式如下（只测试一个特征的一个阈值）：

$$\operatorname{Gini}(D, k)= \max_{t \in T_k} \left( \frac{|D_{\text{left}}^t|}{|D|} \operatorname{Gini}(D_{\text{left}}^t) + \frac{|D_{\text{right}}^t|}{|D|} \operatorname{Gini}(D_{\text{right}}^t) \right)$$

其中 $T_k$ 表示平均值集合，$D_{\text{left/right}}^t$ 表示左右子节点拥有的样本集（左侧小于阈值，右侧大于阈值）。

划分的目标是使得样本集按照某个特征的某个阈值划分之后的纯度提高，因此可找到最合适的划分特征及阈值，如下：

$$k_{\text{best}}, t_{\text{best}} = \underset{k, t}{\operatorname{argmin}} \operatorname{Gini}(D, k, t)$$

##### 3. 均方误差（MSE）

使用前两种指标的决策树可以用来做分类问题，而想要让决策树可以用来做回归问题，就需要不同的指标来决定划分的特征。

这个指标就是均方误差（MSE），其定义式如下：

$$\operatorname{MSE}(D) = \frac{1}{N}\sum_{i = 1}^{N}{\left(y_{i} - \hat{y}\right)^{2}}$$

其中 $D$ 为样本集；$i$ 表示样本集中的第 $i$ 个样本；$N$ 为样本个数；$y_i$ 表示第 $i$ 个样本的标签值；$\hat{y}$ 表示样本标签的均值。

$\operatorname{MSE}(D)$ 的值越小，样本集的值越集中，将这些值视为一类也就更加合理。

对于回归问题，样本的标签是一个连续的变量，因此很大程度上各个样本的标签都不相等，使用上面两种度量方式就可能导致计算出来的纯度指标并没有参考性（都差不多且纯度很低）.

同样的，在中间节点我们可以根据变量的值将样本集划分为多个子集（一般为两个），以节点的 MSE 作为最优划分的度量标准。

其定义如下：

$$\operatorname{MSE}(D, k)= \sum_{j = 1}^{M} \frac{\left| D^{j} \right|}{|D|} \operatorname{MSE} \left(D^{j} \right)$$

其中 $D$ 表示样本集，$k$ 表示离散属性，$M$ 表示离散属性 $k$ 所有可能取值的数量，$D^j$ 表示样本集中第 $j$ 种取值的子样本集。

$operatorname{MSE}(D, k)$ 的值越小，决策树对样本集的拟合程度越高。因此，可找到最合适的划分特征及阈值如下：

$$k_{\text{best}}, t_{\text{best}} = \underset{k, t}{\operatorname{argmin}} \operatorname{MSE}(D, k, t)$$